# Phase 2 Model Validation

## Objective:

Following the comprehensive analysis in Phase 1, `mobilenet_v2` was identified as the optimal architecture, uniquely satisfying our requirements for real-time inference speed, compact size, and strong accuracy potential.

This notebook executes Phase 2 of our project. Instead of re-training the model from scratch, we will adopt a highly efficient strategy: we will acquire and validate a `mobilenet_v2` model that has already been fine-tuned on the full Food101 dataset by the community on the Hugging Face Hub.

The primary goal is to perform a final, rigorous benchmark to confirm that this pre-trained model meets our production criteria before proceeding with deployment.

Our investigation will proceed in two key steps:

1/ Model Acquisition:
> We will first download the selected pre-trained [mobilenet-finetuned-food101](https://huggingface.co/paolinox/mobilenet-finetuned-food101/tree/main) model and prepare the full Food101 test `DataLoader`.

2/ Quick Validation:
> We will then programmatically evaluate the model's final **accuracy**, **size**, and **inference speed** to ensure it aligns with our expectations and is ready for deployment.

## 00. Import base libraries

In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import torch
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
from pathlib import Path

from src.utils import count_loaded_model_parameters, benchmark_loaded_model_speed
from src.engine import evaluate_model

import warnings
# The warnings from Hugging Face are informative but not critical for our task.
# We can safely ignore them to keep the notebook output tidy.
warnings.filterwarnings("ignore")
import json

## 01. Model Acquisition

In this section, we will prepare all necessary components for our final validation. This involves defining the Hugging Face Hub identifier for our chosen [mobilenet-finetuned-food101](https://huggingface.co/paolinox/mobilenet-finetuned-food101/tree/main) model.

We will leverage the `transformers` library to handle the entire acquisition process. Its `AutoModel` and `AutoImageProcessor` classes will automatically download and cache the model's architecture, weights, and the specific preprocessing configuration required. This approach ensures perfect compatibility and reproducibility.

Finally, to establish a robust ground truth for our benchmark, we will use the acquired image processor to create the **full Food101 test** `DataLoader`.

In [2]:
# Setup device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Define the root data directory
DATA_PATH = Path("../data/raw")
DATA_PATH.mkdir(parents=True, exist_ok=True)
print(f"Data directory: {DATA_PATH}")

# Define the directory to save/cache our final model within our project structure
SAVED_MODELS_PATH = Path("../saved_models")
SAVED_MODELS_PATH.mkdir(parents=True, exist_ok=True)
print(f"Local model cache directory: {SAVED_MODELS_PATH}")

Using device: cuda
Data directory: ..\data\raw
Local model cache directory: ..\saved_models


In [3]:
# Define the path for our standard label map
LABEL_MAP_PATH = SAVED_MODELS_PATH / "food101_label_map.json"

# --- Create the standard Food101 label map IF it doesn't exist ---
if not LABEL_MAP_PATH.exists():
    print(f"Standard label map not found. Creating one at: {LABEL_MAP_PATH}")
    
    # Load the dataset just to get the class names in alphabetical order
    temp_dataset = datasets.Food101(root=DATA_PATH, split="test", download=True)
    class_names = temp_dataset.classes # This is a list of 101 class names
    
    # Create a dictionary mapping from integer index to class name
    # e.g., {0: "apple_pie", 1: "baby_back_ribs", ...}
    label_map = {str(i): name for i, name in enumerate(class_names)}
    
    # Save the dictionary to a JSON file
    with open(LABEL_MAP_PATH, "w") as f:
        json.dump(label_map, f, indent=4)
        
    print("-> Successfully created and saved food101_label_map.json")
else:
    print(f"Found existing standard label map at: {LABEL_MAP_PATH}")

Standard label map not found. Creating one at: ..\saved_models\food101_label_map.json
-> Successfully created and saved food101_label_map.json


In [4]:
# --- Configuration for the Chosen Model ---
FINAL_MODEL_INFO = {
    "hub_id": "Haaaaaaaaaax/efficientnet-b3-finetuned-food101",
    "local_dir_name": "Haaaaaaaaaax_efficientnet_b3_finetuned_food101",
}

print(f"Selected model for validation: {FINAL_MODEL_INFO['hub_id']}")

Selected model for validation: Haaaaaaaaaax/efficientnet-b3-finetuned-food101


In [5]:
from transformers import AutoModelForImageClassification, AutoImageProcessor

# The full path to our local model directory
local_model_dir = SAVED_MODELS_PATH / FINAL_MODEL_INFO['local_dir_name']
hub_id = FINAL_MODEL_INFO['hub_id']

# --- Step 1: Check for local project cache or download from Hub ---
if not local_model_dir.exists():
    print(f"Local model not found at '{local_model_dir}'.")
    print(f"Downloading model and processor from Hub: {hub_id}...")
    
    # Use transformers to download the model and processor
    model_from_hub = AutoModelForImageClassification.from_pretrained(hub_id)
    processor_from_hub = AutoImageProcessor.from_pretrained(hub_id)
    
    # Now, save them to our own project's `saved_models` directory
    print(f"Saving model and processor to our local project cache...")
    model_from_hub.save_pretrained(local_model_dir)
    processor_from_hub.save_pretrained(local_model_dir)
    
    print("Download and local caching complete.")
else:
    print(f"Found existing local model at: {local_model_dir}")

# --- Step 2: Load the model and processor FROM OUR LOCAL DIRECTORY ---
print("\nLoading model and processor from local project directory...")

final_model = AutoModelForImageClassification.from_pretrained(local_model_dir)
final_model.to(device)
final_model.eval()

image_processor = AutoImageProcessor.from_pretrained(local_model_dir)
print("-> Model and Image Processor loaded successfully.")

# --- 3. Create a Transform Function and the Final DataLoader ---
def create_transform(processor):
    def transform_func(image):
        if image.mode != "RGB":
            image = image.convert("RGB")
        return processor(image, return_tensors="pt")['pixel_values'].squeeze(0)
    return transform_func

final_transform = create_transform(image_processor)


Local model not found at '..\saved_models\Haaaaaaaaaax_efficientnet_b3_finetuned_food101'.


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Saving model and processor to our local project cache...
Download and local caching complete.

Loading model and processor from local project directory...
-> Model and Image Processor loaded successfully.


In [9]:
# --- 4. Create the Dataset and DataLoader ---
print("\nPreparing the full Food101 test dataset...")
test_dataset = datasets.Food101(root=DATA_PATH,
                                split="test",
                                transform=final_transform,
                                download=True)
print(f"-> Found {len(test_dataset)} images in the test set.")

test_dataloader = DataLoader(dataset=test_dataset,
                             batch_size=32,
                             num_workers=0,
                             shuffle=False)
print(f"-> Final test DataLoader created.")


# --- Final Status ---
print("\n--- Acquisition & Setup Complete ---")
print("The following variables are now ready for the validation section:")
print(f"1. The instantiated model: `final_model` (loaded from '{local_model_dir.name}')")
print(f"2. The validation DataLoader: `test_dataloader`")
print(f"3. The standard label map path: `LABEL_MAP_PATH`")


Preparing the full Food101 test dataset...
-> Found 25250 images in the test set.
-> Final test DataLoader created.

--- Acquisition & Setup Complete ---
The following variables are now ready for the validation section:
1. The instantiated model: `final_model` (loaded from 'Haaaaaaaaaax_efficientnet_b3_finetuned_food101')
2. The validation DataLoader: `test_dataloader`
3. The standard label map path: `LABEL_MAP_PATH`


## 02. Quick Validation

With our `mobilenet_v2` model and the full test DataLoader now ready, we will perform a final, consolidated validation. Unlike the broad exploration in Phase 1, the goal here is to efficiently generate a definitive "report card" for our chosen candidate.
Our streamlined validation process will programmatically compute our three key metrics and present them in a final summary table to confirm the model's suitability for deployment:

- Performance: Calculate the top-1 accuracy on the full test set.
- Efficiency (Size): Calculate the model's exact size in Megabytes.
- Efficiency (Speed): Benchmark the average inference time on a CPU.

In [10]:
# --- 1. Performance Validation: Calculating Accuracy ---
print(f"\nCalculating accuracy for '{FINAL_MODEL_INFO['hub_id']}' using the standard label map...")
final_accuracy = evaluate_model(model=final_model,
                                dataloader=test_dataloader,
                                device=device,
                                label_map_path=LABEL_MAP_PATH,
                                verbose=True)

print(f"\n-> Final Validated Accuracy: {final_accuracy:.2f}%")


Calculating accuracy for 'Haaaaaaaaaax/efficientnet-b3-finetuned-food101' using the standard label map...


Evaluating Accuracy:   0%|          | 0/790 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 03. Phase 2 summary & key takeaways